In [1]:
import os
import argparse
import torch
import torch.nn.parallel
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
import timm
from torch import nn 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import clip
from transformers import CLIPProcessor, CLIPVisionModel, CLIPTextModel, CLIPTokenizer, CLIPModel

In [5]:
CUBattributeclasses=['has a curved (up or down) bill shape',
'has a dagger bill shape',
'has a hooked bill shape',
'has a needle bill shape',
'has a hooked seabird bill shape',
'has a spatulate bill shape',
'has a all-purpose bill shape',
'has a cone bill shape',
'has a specialized bill shape',
'has a blue wing color',
'has a brown wing color',
'has a iridescent wing color',
'has a purple wing color',
'has a rufous wing color',
'has a grey wing color',
'has a yellow wing color',
'has an olive wing color',
'has a green wing color',
'has a pink wing color',
'has an orange wing color',
'has a black wing color',
'has a white wing color',
'has a red wing color',
'has a buff wing color',
'has a blue upperparts color',
'has a brown upperparts color',
'has an iridescent upperparts color',
'has a purple upperparts color',
'has a rufous upperparts color',
'has a grey upperparts color',
'has a yellow upperparts color',
'has a olive upperparts color',
'has a green upperparts color',
'has a pink upperparts color',
'has an orange upperparts color',
'has a black upperparts color',
'has a white upperparts color',
'has a red upperparts color',
'has a buff upperparts color',
'has a blue underparts color',
'has a brown underparts color',
'has an iridescent underparts color',
'has a purple underparts color',
'has a rufous underparts color',
'has a grey underparts color',
'has a yellow underparts color',
'has an olive underparts color',
'has a green underparts color',
'has a pink underparts color',
'has an orange underparts color',
'has a black underparts color',
'has a white underparts color',
'has a red underparts color',
'has a buff underparts color',
'has a solid breast pattern',
'has a spotted breast pattern',
'has a striped breast pattern',
'has a multi-colored breast pattern',
'has a blue back color',
'has a brown back color',
'has a iridescent back color',
'has a purple back color',
'has a rufous back color',
'has a grey back color',
'has a yellow back color',
'has a olive back color',
'has a green back color',
'has a pink back color',
'has an orange back color',
'has a black back color',
'has a white back color',
'has a red back color',
'has a buff back color',
'has a forked tail', 
'has a rounded tail',
'has a notched tail',
'has a fan-shaped tail',
'has a pointed tail',
'has a squared tail',
'has a blue upper tail',
'has a brown upper tail',
'has a iridescent upper tail',
'has a purple upper tail',
'has a rufous upper tail',
'has a grey upper tail',
'has a yellow upper tail',
'has a olive upper tail',
'has a green upper tail',
'has a pink upper tail',
'has a orange upper tail',
'has a black upper tail',
'has a white upper tail',
'has a red upper tail',
'has a buff upper tail',
'has a spotted head pattern',
'has a malar head pattern',
'has a crested head pattern',
'has a masked head pattern',
'has a unique head pattern',
'has a eyebrow head pattern',
'has a eyering head pattern',
'has a plain head pattern',
'has a eyeline head pattern',
'has a striped head pattern',
'has a capped head pattern',
'has a blue breast color',
'has a brown breast color',
'has a iridescent breast color',
'has a purple breast color',
'has a rufous breast color',
'has a grey breast color',
'has a yellow breast color',
'has a olive breast color',
'has a green breast color',
'has a pink breast color',
'has an orange breast color',
'has a black breast color',
'has a white breast color',
'has a red breast color',
'has a buff breast color',
'has a blue throat color',
'has a brown throat color',
'has a iridescent throat color',
'has a purple throat color',
'has a rufous throat color',
'has a grey throat color',
'has a yellow throat color',
'has a olive throat color',
'has a green throat color',
'has a pink throat color',
'has a orange throat color',
'has a black throat color',
'has a white throat color',
'has a red throat color',
'has a buff throat color',
'has a blue eye color',
'has a brown eye color',
'has a purple eye color',
'has a rufous eye color',
'has a grey eye color',
'has a yellow eye color',
'has a olive eye color',
'has a green eye color',
'has a pink eye color',
'has a orange eye color',
'has a black eye color',
'has a white eye color',
'has a red eye color',
'has a buff eye color',
'has a about the same as head bill length',
'has a longer than head bill length',
'has a shorter than head bill length',
'has a blue forehead color',
'has a brown forehead color',
'has a iridescent forehead color',
'has a purple forehead color',
'has a rufous forehead color',
'has a grey forehead color',
'has a yellow forehead color',
'has a olive forehead color',
'has a green forehead color',
'has a pink forehead color',
'has a orange forehead color',
'has a black forehead color',
'has a white forehead color',
'has a red forehead color',
'has a buff forehead color',
'has a blue under tail color',
'has a brown under tail color',
'has a iridescent under tail color',
'has a purple under tail color',
'has a rufous under tail color',
'has a grey under tail color',
'has a yellow under tail color',
'has a olive under tail color',
'has a green under tail color',
'has a pink under tail color',
'has a orange under tail color',
'has a black under tail color',
'has a white under tail color',
'has a red under tail color',
'has a buff under tail color',
'has a blue nape color',
'has a brown nape color',
'has a iridescent nape color',
'has a purple nape color',
'has a rufous nape color',
'has a grey nape color',
'has a yellow nape color',
'has a olive nape color',
'has a green nape color',
'has a pink nape color',
'has a orange nape color',
'has a black nape color',
'has a white nape color',
'has a red nape color',
'has a buff nape color',
'has a blue belly color',
'has a brown belly color',
'has a iridescent belly color',
'has a purple belly color',
'has a rufous belly color',
'has a grey belly color',
'has a yellow belly color',
'has a olive belly color',
'has a green belly color',
'has a pink belly color',
'has a orange belly color',
'has a black belly color',
'has a white belly color',
'has a red belly color',
'has a buff belly color',
'has a rounded-wings wing shape',
'has a pointed-wings wing shape',
'has a broad-wings wing shape',
'has a tapered-wings wing shape',
'has a long-wings wing shape',
'has a large size',
'has a small size',
'has a very large size',
'has a medium size',
'has a very small size',
'has a upright-perching water-like shape',
'has a chicken-like-marsh shape',
'has a long-legged-like shape',
'has a duck-like shape',
'has a owl-like shape',
'has a gull-like shape',
'has a hummingbird-like shape',
'has a pigeon-like shape',
'has a tree-clinging-like shape',
'has a hawk-like shape',
'has a sandpiper-like shape',
'has a upland-ground-like shape',
'has a swallow-like shape',
'has a perching-like shape',
'has a solid back pattern',
'has a spotted back pattern',
'has a striped back pattern',
'has a multi-colored back pattern',
'has a solid tail pattern',
'has a spotted tail pattern',
'has a striped tail pattern',
'has a multi-colored tail pattern',
'has a solid belly pattern',
'has a spotted belly pattern',
'has a striped belly pattern',
'has a multi-colored belly pattern',
'has a blue primary color',
'has a brown primary color',
'has a iridescent primary color',
'has a purple primary color',
'has a rufous primary color',
'has a grey primary color',
'has a yellow primary color',
'has a olive primary color',
'has a green primary color',
'has a pink primary color',
'has a orange primary color',
'has a black primary color',
'has a white primary color',
'has a red primary color',
'has a buff primary color',
'has a blue leg color',
'has a brown leg color',
'has a iridescent leg color',
'has a purple leg color',
'has a rufous leg color',
'has a grey leg color',
'has a yellow leg color',
'has a olive leg color',
'has a green leg color',
'has a pink leg color',
'has a orange leg color',
'has a black leg color',
'has a white leg color',
'has a red leg color',
'has a buff leg color',
'has a blue bill color',
'has a brown bill color',
'has a iridescent bill color',
'has a purple bill color',
'has a rufous bill color',
'has a grey bill color',
'has a yellow bill color',
'has a olive bill color', 
'has a green bill color',
'has a pink bill color',
'has a orange bill color',
'has a black bill color',
'has a white bill color',
'has a red bill color',
'has a buff bill color',
'has a blue crown color',
'has a brown crown color',
'has a iridescent crown color',
'has a purple crown color',
'has a rufous crown color',
'has a grey crown color',
'has a yellow crown color',
'has a olive crown color',
'has a green crown color',
'has a pink crown color',
'has a orange crown color',
'has a black crown color',
'has a white crown color',
'has a red crown color',
'has a buff crown color',
'has a solid wing pattern',
'has a spotted wing pattern',
'has a striped wing pattern',
'has a multi-colored wing pattern']

In [6]:
i=1
for item in CUBattributeclasses:
    i=i+1
    print(item)

has a curved (up or down) bill shape
has a dagger bill shape
has a hooked bill shape
has a needle bill shape
has a hooked seabird bill shape
has a spatulate bill shape
has a all-purpose bill shape
has a cone bill shape
has a specialized bill shape
has a blue wing color
has a brown wing color
has a iridescent wing color
has a purple wing color
has a rufous wing color
has a grey wing color
has a yellow wing color
has an olive wing color
has a green wing color
has a pink wing color
has an orange wing color
has a black wing color
has a white wing color
has a red wing color
has a buff wing color
has a blue upperparts color
has a brown upperparts color
has an iridescent upperparts color
has a purple upperparts color
has a rufous upperparts color
has a grey upperparts color
has a yellow upperparts color
has a olive upperparts color
has a green upperparts color
has a pink upperparts color
has an orange upperparts color
has a black upperparts color
has a white upperparts color
has a red upperpa

In [7]:
len(CUBattributeclasses)

312

In [8]:
CLIP = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14-336")
inputs = tokenizer([(f"This bird {c}") for c in CUBattributeclasses], padding=True, return_tensors="pt")
txt_features = CLIP.get_text_features(**inputs)
txt_features = txt_features / txt_features.norm(dim=-1, keepdim=True)
txt_features = txt_features.float()

np.save('CUB312text_feature.npy',txt_features.detach().numpy())

In [ ]:
#features, _ = clip.load('ViT-L/14@336px', device) 
CoCoclasses=['person','bicycle','car', 'motorcycle', 'airplane', 'bus', 'train','truck',
         'boat','traffic light','fire hydrant','stop sign','parking meter','bench',
         'bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe',
         'backpack','umbrella','handbag','tie','suitcase','frisbee','skis','snowboard',
         'sports ball','kite','baseball bat','baseball glove','skateboard','surfboard',
         'tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl',
         'banana','apple','sandwich','orange','broccoli','carrot','hot dog','pizza',
         'donut','cake','chair','couch','potted plant','bed','dining table','toilet',
         'tv','laptop','mouse','remote','keyboard','cell phone','microwave','oven',
         'toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear',
         'hair drier','toothbrush']
CLIP = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14-336")
inputs = tokenizer(CoCoclasses, padding=True, return_tensors="pt")
txt_features = CLIP.get_text_features(**inputs)
txt_features = txt_features / txt_features.norm(dim=-1, keepdim=True)
txt_features = txt_features.float()

np.save('CoCo80text_feature_labelonly.npy',txt_features.detach().numpy())

In [17]:
clip.available_models()
device = "cpu"
clip_model, preprocess = clip.load('ViT-L/14@336px', device)
ctx_dim = clip_model.ln_final.weight.shape[0]


n_ctx=4
ctx_dim=768
ctx_vectors = torch.empty(n_ctx, ctx_dim, dtype=torch.float32)
nn.init.normal_(ctx_vectors, std=0.02)
prompt_prefix = " ".join(["X"] * n_ctx)


ctx = nn.Parameter(ctx_vectors)
if ctx.dim() == 2:
    ctx = ctx.unsqueeze(0).expand(80, -1, -1)
        
classnames=['person','bicycle','car', 'motorcycle', 'airplane', 'bus', 'train','truck',
         'boat','traffic light','fire hydrant','stop sign','parking meter','bench',
         'bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe',
         'backpack','umbrella','handbag','tie','suitcase','frisbee','skis','snowboard',
         'sports ball','kite','baseball bat','baseball glove','skateboard','surfboard',
         'tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl',
         'banana','apple','sandwich','orange','broccoli','carrot','hot dog','pizza',
         'donut','cake','chair','couch','potted plant','bed','dining table','toilet',
         'tv','laptop','mouse','remote','keyboard','cell phone','microwave','oven',
         'toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear',
         'hair drier','toothbrush']

prompts = [name + "." for name in classnames]
tokenized_prompts = torch.cat([clip.tokenize(p) for p in prompts])


In [21]:
if ctx.dim() == 2:
    ctx = ctx.unsqueeze(0).expand(80, -1, -1)
ctx.shape
a=tokenized_prompts.argmax(dim=-1)
print(a.shape)

torch.Size([80])


In [3]:
CLIP = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
inputs = tokenizer([(f"a photo of a {c}") for c in CoCoclasses], padding=True, return_tensors="pt")
txt_features = CLIP.get_text_features(**inputs)
txt_features = txt_features / txt_features.norm(dim=-1, keepdim=True)
txt_features = txt_features.float()

np.save('CoCo80text_feature_224.npy',txt_features.detach().numpy())

In [7]:
CLIP = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch16")
inputs = tokenizer([(f"a photo of a {c}") for c in CoCoclasses], padding=True, return_tensors="pt")
txt_features = CLIP.get_text_features(**inputs)
txt_features = txt_features / txt_features.norm(dim=-1, keepdim=True)
txt_features = txt_features.float()

np.save('CoCo80text_feature_vitb16_224.npy',txt_features.detach().numpy())

In [6]:
CLIP = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
print(CLIP)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch16")
inputs = tokenizer([(f"a photo of a {c}") for c in CoCoclasses], padding=True, return_tensors="pt")
txt_features = CLIP.get_text_features(**inputs)
txt_features = txt_features / txt_features.norm(dim=-1, keepdim=True)
txt_features = txt_features.float()

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0): CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05, element

In [7]:
txt_features.shape

torch.Size([80, 512])

In [5]:
CLIP = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
inputs = tokenizer([(f"a photo of a {c}") for c in CoCoclasses], padding=True, return_tensors="pt")
txt_features = CLIP.get_text_features(**inputs)
txt_features = txt_features / txt_features.norm(dim=-1, keepdim=True)
txt_features = txt_features.float()

np.save('CoCo80text_feature_vitb32_224.npy',txt_features.detach().numpy())

In [6]:
txt_features.shape

torch.Size([80, 512])

In [3]:
features = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")

Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.7.layer_norm2.bias', 'text_model.encoder.layers.0.self_attn.q_proj.weight', 'text_model.encoder.layers.2.mlp.fc1.bias', 'text_model.encoder.layers.0.mlp.fc2.bias', 'text_model.encoder.layers.8.self_attn.k_proj.bias', 'text_model.encoder.layers.9.layer_norm2.bias', 'text_model.encoder.layers.10.self_attn.v_proj.bias', 'text_model.encoder.layers.11.self_attn.q_proj.bias', 'text_model.encoder.layers.8.mlp.fc2.weight', 'text_model.encoder.layers.0.self_attn.k_proj.weight', 'text_model.encoder.layers.11.self_attn.q_proj.weight', 'text_model.encoder.layers.3.self_attn.out_proj.bias', 'text_model.encoder.layers.7.layer_norm1.weight', 'text_model.encoder.layers.11.layer_norm1.bias', 'text_model.encoder.layers.9.mlp.fc1.weight', 'text_model.encoder.layers.9.layer_norm2.weight', 'text_model.encoder.layers.5.self_attn.k_proj.weight', 'text_model.encoder

In [4]:
features

CLIPVisionModel(
  (vision_model): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
      (position_embedding): Embedding(50, 768)
    )
    (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0): CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2